# Лабораторная работа №8


## 1. Выбор начальных условий
a.	Выбрать набор данных для задачи классификации и обосновать его выбор.
Датасет - https://www.kaggle.com/datasets/ronakgohil/license-plate-dataset

Данный датасет с изображениями автомобильных номерных знаков актуален для задач автоматического распознавания номерных знаков, что важно для систем контроля транспорта, безопасности на дорогах и управления парковками.

b.	Выбрать метрики качества и обосновать их выбор.


* mAP\@0.5 — измеряет, насколько модель правильно находит номера с допуском локализации (IoU=0.5).
* mAP@\[0.5:0.95] — средний показатель качества детекции при разных уровнях точности локализации.
* Precision — доля правильных номеров среди всех найденных. Важно избегать ложных срабатываний.
* Recall — доля найденных номеров из всех реально присутствующих. Важно не пропускать номера.




## 2.	Создание бейзлайна и оценка качества


In [1]:
!pip install kaggle --quiet
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━ 474.6/664.8 MB 162.4 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.4 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please u

In [7]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00


In [24]:
!pip install torchmetrics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 57.1 MB/s eta 0:00:00


Загрузим датасет

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ketrum","key":"4f7898fa6bc17d128bca596c0fb2323f"}'}

In [4]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')

In [5]:
!kaggle datasets download -d ronakgohil/license-plate-dataset
!unzip -q license-plate-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ronakgohil/license-plate-dataset
License(s): CC0-1.0
 77% 114M/147M [00:00<00:00, 1.19GB/s]
100% 147M/147M [00:00<00:00, 821MB/s] 


### a.	Обучить модель для выбранного набора данных

In [8]:
from ultralytics import YOLO

yolo = YOLO('yolo11n.pt')
yaml_path = 'archive/dataset.yaml'

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 349MB/s]


In [13]:
res_yolo = yolo.train(
    data=yaml_path,
    epochs=5,
    imgsz=224,
    batch=32,
    name='license-plate'
)

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

100%|██████████| 755k/755k [00:00<00:00, 97.2MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 13.5±4.9 MB/s, size: 24.6 KB)


train: Scanning /content/archive/labels/train... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:01<00:00, 814.45it/s]

...

        5/5     0.838G     0.9895     0.6903      0.907         39        224: 100%|██████████| 48/48 [00:10<00:00,  4.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.23it/s]

                   all        169        169      0.987      0.929      0.981       0.74



5 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/license-plate5/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/license-plate5/weights/best.pt, 5.4MB

Validating runs/detect/license-plate5/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.10it/s]


                   all        169        169      0.987      0.929      0.981      0.739
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/license-plate5


### b.	Оценить качество модели по выбранным метрикам на выбранном наборе данных


In [15]:
results_val_base = yolo.val(data=yaml_path, iou=0.5)
print("Метрики:")
print("mAP@0.5:", results_val_base.box.map50)
print("mAP@0.5:0.95:", results_val_base.box.map)
print("Precision:", results_val_base.box.p)
print("Recall:", results_val_base.box.r)


Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2719.8±995.2 MB/s, size: 204.2 KB)


val: Scanning /content/archive/labels/val.cache... 169 images, 0 backgrounds, 0 corrupt: 100%|██████████| 169/169 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.61it/s]


                   all        169        169      0.987      0.929      0.981      0.739
Speed: 0.5ms preprocess, 4.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/license-plate52
Метрики:
mAP@0.5: 0.98132428952884
mAP@0.5:0.95: 0.7391155052513505
Precision: [    0.98742]
Recall: [    0.92866]


## 3.	Улучшение бейзлайна

### a. Сформулировать гипотезы (аугментации данных, подбор моделей, подбор гиперпараметров и т.д.)

Проверим следующие гипотезы; 1) увеличение количества эпох 2) аугментация 3) увеличение скорости обучения 4) использование модели yolo11s

### b. Проверить гипотезы

1) увеличение количества эпох

In [16]:
res_yolo = yolo.train(
    data=yaml_path,
    epochs=10,
    imgsz=224,
    batch=32,
    name='license-plate'
)

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

train: Scanning /content/archive/labels/train.cache... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:00<?, ?it/s]

train: /content/archive/images/train/car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 396.9±233.0 MB/s, size: 162.0 KB)


val: Scanning /content/archive/labels/val.cache... 169 images, 0 backgrounds, 0 corrupt: 100%|██████████| 169/169 [00:00<?, ?it/s]


...

      10/10     0.752G     0.8216     0.5085     0.8637         22        224: 100%|██████████| 48/48 [00:11<00:00,  4.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.35it/s]

                   all        169        169      0.988       0.97      0.987      0.779



10 epochs completed in 0.034 hours.
Optimizer stripped from runs/detect/license-plate6/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/license-plate6/weights/best.pt, 5.4MB

Validating runs/detect/license-plate6/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.92it/s]


                   all        169        169      0.982      0.964      0.987       0.78
Speed: 0.0ms preprocess, 0.9ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/license-plate6


2) аугментация

In [17]:
res_yolo = yolo.train(
    data=yaml_path,
    epochs=5,
    imgsz=224,
    batch=32,
    augment=True,
    name='license-plate'
)

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /content/archive/labels/train.cache... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:00<?, ?it/s]

train: /content/archive/images/train/car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 454.2±118.6 MB/s, size: 162.0 KB)


val: Scanning /content/archive/labels/val.cache... 169 images, 0 backgrounds, 0 corrupt: 100%|██████████| 169/169 [00:00<?, ?it/s]


...

        5/5     0.789G     0.8912     0.5495      0.884         39        224: 100%|██████████| 48/48 [00:11<00:00,  4.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.27it/s]

                   all        169        169      0.968      0.976      0.985      0.766



5 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/license-plate7/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/license-plate7/weights/best.pt, 5.4MB

Validating runs/detect/license-plate7/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.50it/s]


                   all        169        169      0.946      0.942      0.981      0.754
Speed: 0.1ms preprocess, 7.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/license-plate7


 3) увеличение скорости обучения

In [19]:
res_yolo = yolo.train(
    data=yaml_path,
    lr0=0.01,
    epochs=5,
    imgsz=224,
    batch=32,

    name='license-plate'
)

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /content/archive/labels/train.cache... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:00<?, ?it/s]

train: /content/archive/images/train/car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 518.5±100.3 MB/s, size: 162.0 KB)


val: Scanning /content/archive/labels/val.cache... 169 images, 0 backgrounds, 0 corrupt: 100%|██████████| 169/169 [00:00<?, ?it/s]


...

        5/5      0.83G     0.8448     0.5114     0.8704         39        224: 100%|██████████| 48/48 [00:11<00:00,  4.36it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.36it/s]

                   all        169        169      0.971      0.981      0.989      0.785



5 epochs completed in 0.017 hours.
Optimizer stripped from runs/detect/license-plate9/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/license-plate9/weights/best.pt, 5.4MB

Validating runs/detect/license-plate9/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.92it/s]


                   all        169        169      0.968      0.947      0.986      0.774
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/license-plate9


 4) использование модели yolo11s

In [20]:
yolo = YOLO('yolo11s.pt')
res_yolo = yolo.train(
    data=yaml_path,
    epochs=5,
    imgsz=224,
    batch=32,
    name='license-plate'
)

100%|██████████| 18.4M/18.4M [00:00<00:00, 437MB/s]

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate10, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

  8                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  9                  -1  1    656896  ultralytics.nn.modules.block.SPPF            [512, 512, 5]                 
 10                  -1  1    990976  ultralytics.nn.modules.block.C2PSA           [512, 512, 1]                 
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  1    443776  ultralytics.nn.modules.block.C3k2            [768, 256, 1, False]          
 14                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 15             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 16                  -1  1    127680  ultralytics.nn.modules.block.C3k2            [512,

train: Scanning /content/archive/labels/train.cache... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:00<?, ?it/s]

train: /content/archive/images/train/car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

        5/5      1.37G     0.9178     0.5574     0.9085         39        224: 100%|██████████| 48/48 [00:10<00:00,  4.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.36it/s]

                   all        169        169      0.979       0.97      0.991      0.768



5 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/license-plate10/weights/last.pt, 19.1MB
Optimizer stripped from runs/detect/license-plate10/weights/best.pt, 19.1MB

Validating runs/detect/license-plate10/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.44it/s]


                   all        169        169      0.979       0.97      0.991       0.77
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/detect/license-plate10


###  c. Сформировать улучшенный бейзлайн по результатам проверки гипотез

В результате проверки гипотез, все гипотезы показали улучшения. Улучшенный бейзлайн включает в себя: 1) увеличение количества эпох 2) аугментация  3) увеличение скорости обучения 4) использование модели yolo11s

### d. Обучить модели с улучшенным бейзлайном на выбранном наборе данных

In [23]:
yolo = YOLO('yolo11s.pt')
res_yolo = yolo.train(
    data=yaml_path,
    lr0=0.01,
    epochs=10,
    imgsz=224,
    batch=16,
    augment=True,
    name='license-plate'
)

Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=archive/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=license-plate13, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretraine

train: Scanning /content/archive/labels/train.cache... 1526 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1526/1526 [00:00<?, ?it/s]

train: /content/archive/images/train/car-wbs-MH03AR5549_00000.jpg: corrupt JPEG restored and saved
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 419.6±155.5 MB/s, size: 162.0 KB)


val: Scanning /content/archive/labels/val.cache... 169 images, 0 backgrounds, 0 corrupt: 100%|██████████| 169/169 [00:00<?, ?it/s]


...

      10/10      4.46G     0.7741     0.4343     0.8721          6        224: 100%|██████████| 96/96 [00:14<00:00,  6.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.65it/s]

                   all        169        169      0.976      0.976      0.988      0.799



10 epochs completed in 0.045 hours.
Optimizer stripped from runs/detect/license-plate13/weights/last.pt, 19.1MB
Optimizer stripped from runs/detect/license-plate13/weights/best.pt, 19.1MB

Validating runs/detect/license-plate13/weights/best.pt...
Ultralytics 8.3.135 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.52it/s]


                   all        169        169      0.974       0.97      0.987      0.792
Speed: 0.0ms preprocess, 9.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/license-plate13


### e. Оценить качество моделей с улучшенным бейзлайном по выбранным метрикам на выбранном наборе данных

Precision =  0.974
Recall = 0.97
mAP50 = 0.987
mAP50-95 = 0.792

### f. Сравнить результаты моделей с улучшенным бейзлайном в сравнении с результатами из пункта 2

Бейзлайн
Precision = 0.987
Recall = 0.929
mAP50 = 0.981
mAP50-95 = 0.739

Улучшенный бейзлайн
Precision =  0.974
Recall = 0.97
mAP50 = 0.987
mAP50-95 = 0.792

### g. Сделать выводы

С помощью улучшеного бейзлайна удалось совсем немного повысить качество модели

## 4. Имплементация алгоритма машинного обучения

### a. Самостоятельно имплементировать модель машинного обучения

In [55]:
import os, yaml, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [56]:
train_img_dir = 'archive/images/train/'
val_img_dir = 'archive/images/val/'
train_label_dir = 'archive/labels/train/'
val_label_dir = 'archive/labels/val/'
classes = ['license_plate']
num_classes = len(classes)

In [80]:
class MyYoloDataset(Dataset):
    def __init__(self, img_dir, label_dir, img_size=224, S=7, B=1, C=20, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.lower().endswith(('jpg','jpeg','png'))]
        self.img_size = img_size
        self.S = S
        self.B =B
        self.C=C
        self.transform = transform if transform else T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
        ])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        label_path = os.path.join(self.label_dir, os.path.splitext(self.img_files[idx])[0] + '.txt')
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        target = torch.zeros((self.S, self.S, 5*self.B + self.C))
        boxes = []
        labels = []

        if os.path.exists(label_path):
            for line in open(label_path, 'r').read().splitlines():
                cls, x_ctr, y_ctr, w, h = map(float, line.split())
                cls = int(cls)
                i = int(x_ctr * self.S)
                j = int(y_ctr * self.S)
                x_cell = x_ctr * self.S - i
                y_cell = y_ctr * self.S - j
                if target[j, i, 4] == 0:
                    target[j, i, 0] = x_cell
                    target[j, i, 1] = y_cell
                    target[j, i, 2] = w
                    target[j, i, 3] = h
                    target[j, i, 4] = 1
                    target[j, i, 5 + cls] = 1
                x_abs = x_ctr * self.img_size
                y_abs = y_ctr * self.img_size
                w_abs = w * self.img_size
                h_abs = h * self.img_size
                x1 = x_abs - w_abs/2
                y1 = y_abs - h_abs/2
                x2 = x_abs + w_abs/2
                y2 = y_abs + h_abs/2
                boxes.append([x1, y1, x2, y2])
                labels.append(cls)

        boxes  = torch.tensor(boxes, dtype=torch.float32) if boxes  else torch.zeros((0,4), dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,), dtype=torch.int64)
        return img, target, boxes, labels

In [81]:
class MyYolo(nn.Module):
    def __init__(self, S=7, B=1, C=20):
        super().__init__()
        self.S = S
        self.B = B
        self.C = C

        self.backbon = nn.Sequential(
            nn.Conv2d(3,16, 3, 1, 1), nn.BatchNorm2d(16), nn.LeakyReLU(0.1), nn.MaxPool2d(2,2),
            nn.Conv2d(16, 32, 3, 1, 1), nn.BatchNorm2d(32), nn.LeakyReLU(0.1), nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.1), nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.1), nn.MaxPool2d(2,2),
            nn.Conv2d(128, 256,3, 1, 1), nn.BatchNorm2d(256), nn.LeakyReLU(0.1), nn.MaxPool2d(2,2),
            )
        self.pred = nn.Conv2d(256, self.B*5 + self.C, 1)

    def forward(self, x):
        x =self.backbon(x)
        x =self.pred(x)
        x =x.permute(0, 2, 3, 1)
        return x



In [82]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=1, C=20, lambda_coord=5, lambda_noobj=0.5):
        super().__init__()
        self.S = S
        self.B = B
        self.C = C
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        self.mse = nn.MSELoss(reduction='sum')
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, preds, target):
        obj_mask = target[..., 4]
        noobj_mask = 1-obj_mask

        pred_box = torch.sigmoid(preds[..., :4])
        pred_obj = preds[..., 4]
        pred_cls = preds[..., 5:]
        tgt_box = target[..., :4]
        tgt_obj = target[..., 4]
        tgt_cls_oh = target[..., 5:]

        coord_loss = self.mse(obj_mask.unsqueeze(-1) * pred_box,obj_mask.unsqueeze(-1)*tgt_box)
        obj_loss = torch.sum(obj_mask* self.bce(pred_obj, tgt_obj))
        noobj_loss = torch.sum(noobj_mask *self.bce(pred_obj, tgt_obj))

        pred_cls_flat = pred_cls.reshape(-1, self.C)
        tgt_cls_ids = torch.argmax(tgt_cls_oh, dim=-1).reshape(-1)
        cls_loss_all = self.ce(pred_cls_flat, tgt_cls_ids)
        cls_loss = torch.sum(obj_mask.reshape(-1) * cls_loss_all)

        total_loss = (self.lambda_coord * coord_loss + obj_loss + self.lambda_noobj * noobj_loss + cls_loss)
        return total_loss



In [86]:
from tqdm import tqdm

def train_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for imgs, y_targets, _, _ in loader:
        imgs = imgs.to(device)
        y_targets = y_targets.to(device)
        preds = model(imgs)
        loss = loss_fn(preds, y_targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print("loss: ", total_loss / len(loader))

In [97]:
def collate_fn(batch):
    imgs = torch.stack([item[0] for item in batch], dim=0)
    yolo_targets = torch.stack([item[1] for item in batch], dim=0)
    boxes_list = [item[2] for item in batch]
    labels_list = [item[3] for item in batch]
    return imgs, yolo_targets, boxes_list, labels_list

batch_size = 16
img_size = 224
train_dataset = MyYoloDataset(train_img_dir, train_label_dir, 224, 7, 1, 1)
val_dataset = MyYoloDataset(val_img_dir, val_label_dir,  224, 7, 1, 1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, collate_fn=collate_fn)



### b. Обучить имплементированную модель на выбранном наборе данных

In [ ]:
model = MyYolo(7, 1, 1).to(device)
loss_fn = YoloLoss(7, 1, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 5

for epoch in range(1, epochs+1):
    print(f"Epoch [{epoch}/{epochs}]")
    train_epoch(train_loader, model, optimizer,loss_fn, device)

Epoch [1/5]
loss:  82.29254643122356
Epoch [2/5]
loss:  40.52777428428332
Epoch [3/5]
loss:  30.293502748012543
Epoch [4/5]
loss:  24.111383110284805
Epoch [5/5]
loss:  19.846983442703884


### c. Оценить качество имплементированной модели по выбранным метрикам на выбранном наборе данных

In [ ]:
model.eval()
map50_metric = MeanAveragePrecision(iou_thresholds=[0.5])
map50_95_metric = MeanAveragePrecision()
with torch.no_grad():
    for imgs, _, boxes_l, labels_l in val_loader:
        imgs = imgs.to(device)
        preds = model(imgs).cpu()

        for b in range(imgs.size(0)):
            det_boxes = []
            det_scores = []
            det_labels = []
            for i in range(S):
                for j in range(S):
                    conf = torch.sigmoid(preds[b,i,j,4])
                    if conf < 0.5:
                        continue
                    x_cell, y_cell, w_cell, h_cell = torch.sigmoid(preds[b, i, j, :4])
                    x_center = (j + x_cell)/S
                    y_center = (i + y_cell)/S
                    w = w_cell
                    h = h_cell

                    x1 = (x_center-w/2) * img_size
                    y1 = (y_center-h/2) * img_size
                    x2 = (x_center + w/2) * img_size
                    y2 = (y_center + h/2) * img_size
                    det_boxes.append([x1, y1, x2, y2])

                    cls_probs = torch.softmax(preds[b, i, j, 5:], dim=-1)
                    cls_score, cls_idx = torch.max(cls_probs, dim=-1)
                    det_scores.append((conf * cls_score).item())
                    det_labels.append(cls_idx.item())

            if len(det_boxes) > 0:
                batch_pred = {
                    'boxes': torch.tensor(det_boxes, dtype=torch.float32),
                    'scores': torch.tensor(det_scores, dtype=torch.float32),
                    'labels': torch.tensor(det_labels, dtype=torch.int64)
                }
            else:
                batch_pred = {
                    'boxes': torch.zeros((0, 4), dtype=torch.float32),
                    'scores': torch.zeros((0,), dtype=torch.float32),
                    'labels': torch.zeros((0,), dtype=torch.int64)
                }

            gt = {
                'boxes': boxes_l[b].to(torch.float32),
                'labels': labels_l[b]
            }
            map50_metric.update([batch_pred], [gt])
            map50_95_metric.update([batch_pred], [gt])

res50 = map50_metric.compute()
res50_95 = map50_95_metric.compute()

print(f"mAP50: {res50['map_50'].item():.4f}")
print(f"mAP50-95: {res50_95['map'].item():.4f}")


mAP50: 0.2008
mAP50-95: 0.0540


### d. Сравнить результаты имплементированных моделей в сравнении с результатами из пункта 2

Бейзлайн
mAP50 = 0.981
mAP50-95 = 0.739

Собственный бейзлайн
mAP50: 0.2008
mAP50-95: 0.0540

### e. Сделать выводы

Собственная реализация показывает результаты значительно хуже

### f. Добавить техники из улучшенного бейзлайна (пункт 3с)

Увеличим количество эпох

In [100]:
batch_size = 16
img_size = 224
train_dataset = MyYoloDataset(train_img_dir, train_label_dir, 224, 7, 1, 1)
val_dataset = MyYoloDataset(val_img_dir, val_label_dir,  224, 7, 1, 1)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True, collate_fn=collate_fn)

### g. Обучить модели для выбранных наборов данных

In [101]:
model = MyYolo(7, 1, 1).to(device)
loss_fn = YoloLoss(7, 1, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 10

for epoch in range(1, epochs+1):
    print(f"Epoch [{epoch}/{epochs}]")
    train_epoch(train_loader, model, optimizer,loss_fn, device)

Epoch [1/10]
loss:  90.38649453719457
Epoch [2/10]
loss:  44.86075961589813
Epoch [3/10]
loss:  33.08886554837227
Epoch [4/10]
loss:  26.19371524453163
Epoch [5/10]
loss:  21.67073021332423
Epoch [6/10]
loss:  18.313269793987274
Epoch [7/10]
loss:  15.374690492947897
Epoch [8/10]
loss:  13.160887469847998
Epoch [9/10]
loss:  11.133304019769033
Epoch [10/10]
loss:  9.440500766038895


### h. Оценить качество моделей по выбранным метрикам на выбранном наборе данных

In [102]:
model.eval()
map50_metric = MeanAveragePrecision(iou_thresholds=[0.5])
map50_95_metric = MeanAveragePrecision()
with torch.no_grad():
    for imgs, _, boxes_l, labels_l in val_loader:
        imgs = imgs.to(device)
        preds = model(imgs).cpu()

        for b in range(imgs.size(0)):
            det_boxes = []
            det_scores = []
            det_labels = []
            for i in range(S):
                for j in range(S):
                    conf = torch.sigmoid(preds[b,i,j,4])
                    if conf < 0.5:
                        continue
                    x_cell, y_cell, w_cell, h_cell = torch.sigmoid(preds[b, i, j, :4])
                    x_center = (j + x_cell)/S
                    y_center = (i + y_cell)/S
                    w = w_cell
                    h = h_cell

                    x1 = (x_center-w/2) * img_size
                    y1 = (y_center-h/2) * img_size
                    x2 = (x_center + w/2) * img_size
                    y2 = (y_center + h/2) * img_size
                    det_boxes.append([x1, y1, x2, y2])

                    cls_probs = torch.softmax(preds[b, i, j, 5:], dim=-1)
                    cls_score, cls_idx = torch.max(cls_probs, dim=-1)
                    det_scores.append((conf * cls_score).item())
                    det_labels.append(cls_idx.item())

            if len(det_boxes) > 0:
                batch_pred = {
                    'boxes': torch.tensor(det_boxes, dtype=torch.float32),
                    'scores': torch.tensor(det_scores, dtype=torch.float32),
                    'labels': torch.tensor(det_labels, dtype=torch.int64)
                }
            else:
                batch_pred = {
                    'boxes': torch.zeros((0, 4), dtype=torch.float32),
                    'scores': torch.zeros((0,), dtype=torch.float32),
                    'labels': torch.zeros((0,), dtype=torch.int64)
                }

            gt = {
                'boxes': boxes_l[b].to(torch.float32),
                'labels': labels_l[b]
            }
            map50_metric.update([batch_pred], [gt])
            map50_95_metric.update([batch_pred], [gt])

res50 = map50_metric.compute()
res50_95 = map50_95_metric.compute()

print(f"mAP50: {res50['map_50'].item():.4f}")
print(f"mAP50-95: {res50_95['map'].item():.4f}")


mAP50: 0.3018
mAP50-95: 0.0793


### i. Сравнить результаты моделей в сравнении с результатами из пункта 3



Улучшенный бейзлайн
mAP50 = 0.987
mAP50-95 = 0.792


Собственный улучшенный бейзлайн
mAP50: 0.3018
mAP50-95: 0.0793

###  j. Сделать выводы

Собственная реализация показывает результаты значительно хуже, так как архитектура  Yolo довольно сложна, увеличение эпох немного улучшило качество собственной модели